# Pear Instance Segmentation with COCO dataformat
<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

## Install detectron2

> **Important**: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md). 

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.
print('Installation ')

## Setup

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
!wget https://smartpears.s3.eu-west-1.amazonaws.com/Pears.zip
!unzip Pears.zip
!ls

## Register data-set

In order to use a dataset with Detectron2 we need to register it. For more information check out the official documentation.

You can change the single_flag variable to use the single or multiple pear dataset.

In [ ]:
from detectron2.data.datasets import register_coco_instances

#bool to switch between single and multiple pears dataset
single_flag = False


for d in ["train", "test"]:
    if single_flag == True:
        register_coco_instances(f"Pears_Single_{d}", {}, f"Pears/Single/{d}.json", f"Pears/Single/{d}")
    else:
        register_coco_instances(f"Pears_Multiple_{d}", {}, f"Pears/Multiple/{d}.json", f"Pears/Multiple/{d}")

In [ ]:
import random
from detectron2.data import DatasetCatalog, MetadataCatalog


if single_flag == True:
    dataset_dicts = DatasetCatalog.get("Pears_Single_train")
    peer_metadata = MetadataCatalog.get("Pears_Single_train")
else:
    dataset_dicts = DatasetCatalog.get("Pears_Multiple_train")
    peer_metadata = MetadataCatalog.get("Pears_Multiple_train")

for d in random.sample(dataset_dicts, 4):
    #print('afbeelding d', d)

    seg_array = d['annotations'][0]['segmentation'][0]
    #print(seg_array)
    re_array = []
    #print(len(seg_array))
    for i in range(len(seg_array)):
        #print(seg_array[i])
        if i % 2 == 0:
            re_array.extend([float((seg_array[i]) *1.00 )- 0])
        else:
            re_array.extend([float((seg_array[i]) *1.00 )+ 0])

        
    #re_array = re_array.reverse()
    #print('re_array', re_array)

    d['annotations'][0]['segmentation'][0] = re_array

    img = cv2.imread(d["file_name"])
    v = Visualizer(img, metadata=peer_metadata, scale=1.0)
    v = v.draw_dataset_dict(d)
    print(v)
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image(), cv2.COLOR_BGR2RGB))
    plt.show()

## Train model

Now, let's fine-tune a pretrained FasterRCNN instance segmentation model on the microcontroller data-set.

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
if single_flag == True: 
    cfg.DATASETS.TRAIN = ("Pears_Single_train",)
else:
    cfg.DATASETS.TRAIN = ("Pears_Multiple_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Use model for inference

Now, we can perform inference on our validation set by creating a predictor object.

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
if single_flag == True: 
    cfg.DATASETS.TEST = ("Pears_Single_test", )
else:
    cfg.DATASETS.TEST = ("Pears_Multiple_test", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
if single_flag == True: 
    dataset_dicts = DatasetCatalog.get("Pears_Single_test")
else:
    dataset_dicts = DatasetCatalog.get("Pears_Multiple_test")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=peer_metadata, 
                   scale=1.0, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    print('filename: ', 'output_' + d["file_name"] )
    cv2.imwrite(d["file_name"].replace('.jpg', '_output.jpg') ,cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()